# Project 4 - Group 3 (Ashrita Surisetti, Ali Yazan, Malini Sintre)
# Introduction

### Clean-up of Dataset A

In [3]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [4]:
df=pd.read_csv('Resources/2019-Nov.csv', encoding='utf-8')
df.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-11-01 00:00:00 UTC,view,1003461,2053013555631882655,electronics.smartphone,xiaomi,489.07,520088904,4d3b30da-a5e4-49df-b1a8-ba5943f1dd33
1,2019-11-01 00:00:00 UTC,view,5000088,2053013566100866035,appliances.sewing_machine,janome,293.65,530496790,8e5f4f83-366c-4f70-860e-ca7417414283
2,2019-11-01 00:00:01 UTC,view,17302664,2053013553853497655,NaN,creed,28.31,561587266,755422e7-9040-477b-9bd2-6a6e8fd97387
3,2019-11-01 00:00:01 UTC,view,3601530,2053013563810775923,appliances.kitchen.washer,lg,712.87,518085591,3bfb58cd-7892-48cc-8020-2f17e6de6e7f
4,2019-11-01 00:00:01 UTC,view,1004775,2053013555631882655,electronics.smartphone,xiaomi,183.27,558856683,313628f1-68b8-460d-84f6-cec7a8796ef2


In [5]:
memory_usage_mb = df.memory_usage(deep=True).sum() / (1024 * 1024)
print("Memory usage of df:", memory_usage_mb, "MB")

Memory usage of df: 25028.222966194153 MB


In [6]:
num_rows = df.shape[0]
num_rows

67501979

In [7]:
#Dropping user_session 
df = df.drop(columns='user_session')
#Dropping No value cells 
df_cleaned = df.dropna()
# Reset index
df_cleaned = df_cleaned.reset_index(drop=True)


In [8]:
num_rows_cleaned = df_cleaned.shape[0]
num_rows_cleaned

42089570

In [9]:
memory_usage_mb = df_cleaned.memory_usage(deep=True).sum() / (1024 * 1024)
print("Memory usage of df_cleaned:", memory_usage_mb, "MB")

Memory usage of df_cleaned: 12652.161519050598 MB


In [10]:
# Converting event_time to datetime
try:
    df_cleaned['event_time'] = pd.to_datetime(df_cleaned['event_time'], format='%Y-%m-%d %H:%M:%S %Z', errors='coerce')
except Exception as e:
    print("Error during conversion:", e)


In [11]:
# Print data types of each column
print("Data types of each column:")
print(df_cleaned.dtypes)
print()

Data types of each column:
event_time       datetime64[ns, UTC]
event_type                    object
product_id                     int64
category_id                    int64
category_code                 object
brand                         object
price                        float64
user_id                        int64
dtype: object



In [12]:
# Renaming columns
df_cleaned = df_cleaned.rename(columns={
    'event_time': 'Timestamp',
    'event_type': 'Visitor_Action',
    'product_id': 'Product',
    'category_id': 'Product_Category',
    'category_code': 'Category_Name',
    'brand': 'Brand',
    'price': 'Price',
    'user_id': 'Visitor_ID'
})
df_cleaned

,Timestamp,Visitor_Action,Product,Product_Category,Category_Name,Brand,Price,Visitor_ID
0,2019-11-01 00:00:00+00:00,view,1003461,2053013555631882655,electronics.smartphone,xiaomi,489.07,520088904
1,2019-11-01 00:00:00+00:00,view,5000088,2053013566100866035,appliances.sewing_machine,janome,293.65,530496790
2,2019-11-01 00:00:01+00:00,view,3601530,2053013563810775923,appliances.kitchen.washer,lg,712.87,518085591
3,2019-11-01 00:00:01+00:00,view,1004775,2053013555631882655,electronics.smartphone,xiaomi,183.27,558856683
4,2019-11-01 00:00:01+00:00,view,1306894,2053013558920217191,computers.notebook,hp,360.09,520772685
...,...,...,...,...,...,...,...,...
42089565,2019-11-30 23:59:57+00:00,view,12301059,2053013556311359947,construction.tools.drill,bosch,266.16,562661595
42089566,2019-11-30 23:59:58+00:00,view,28719425,2053013565639492569,apparel.shoes,baden,62.81,545223467
42089567,2019-11-30 23:59:59+00:00,view,1004833,2053013555631882655,electronics.smartphone,samsung,167.03,557794415
42089568,2019-11-30 23:59:59+00:00,view,2701706,2053013563911439225,appliances.kitchen.refrigerators,samsung,566.27,531607492


In [13]:
# Extract day of the week (Monday=0, Sunday=6) and day of the month
df_cleaned.loc[:, 'Day_of_Week'] = df_cleaned['Timestamp'].dt.dayofweek
df_cleaned.loc[:, 'Day_of_Month'] = df_cleaned['Timestamp'].dt.day

df_cleaned = df_cleaned[['Timestamp','Day_of_Week','Day_of_Month','Visitor_ID','Visitor_Action','Product','Product_Category','Category_Name','Brand','Price']]
df_cleaned

,Timestamp,Day_of_Week,Day_of_Month,Visitor_ID,Visitor_Action,Product,Product_Category,Category_Name,Brand,Price
0,2019-11-01 00:00:00+00:00,4,1,520088904,view,1003461,2053013555631882655,electronics.smartphone,xiaomi,489.07
1,2019-11-01 00:00:00+00:00,4,1,530496790,view,5000088,2053013566100866035,appliances.sewing_machine,janome,293.65
2,2019-11-01 00:00:01+00:00,4,1,518085591,view,3601530,2053013563810775923,appliances.kitchen.washer,lg,712.87
3,2019-11-01 00:00:01+00:00,4,1,558856683,view,1004775,2053013555631882655,electronics.smartphone,xiaomi,183.27
4,2019-11-01 00:00:01+00:00,4,1,520772685,view,1306894,2053013558920217191,computers.notebook,hp,360.09
...,...,...,...,...,...,...,...,...,...,...
42089565,2019-11-30 23:59:57+00:00,5,30,562661595,view,12301059,2053013556311359947,construction.tools.drill,bosch,266.16
42089566,2019-11-30 23:59:58+00:00,5,30,545223467,view,28719425,2053013565639492569,apparel.shoes,baden,62.81
42089567,2019-11-30 23:59:59+00:00,5,30,557794415,view,1004833,2053013555631882655,electronics.smartphone,samsung,167.03
42089568,2019-11-30 23:59:59+00:00,5,30,531607492,view,2701706,2053013563911439225,appliances.kitchen.refrigerators,samsung,566.27


In [14]:
#rounding up the prices - no decimal points
df_cleaned.loc[:,'Price'] = df_cleaned['Price'].round(decimals=0)
# List unique values with counts for categorical columns
for column in df_cleaned.select_dtypes(include='object'):
    print(f"Unique values for {column}:")
    print(df_cleaned[column].value_counts())
    print()

Unique values for Visitor_Action:
Visitor_Action
view        39315226
cart         2115088
purchase      659256
Name: count, dtype: int64

Unique values for Category_Name:
Category_Name
electronics.smartphone          16353582
electronics.video.tv             2195119
computers.notebook               2164657
electronics.clocks               1811325
electronics.audio.headphone      1803895
                                  ...   
apparel.shorts                       447
construction.tools.screw             157
appliances.kitchen.fryer             105
country_yard.furniture.bench           2
apparel.jacket                         1
Name: count, Length: 129, dtype: int64

Unique values for Brand:
Brand
samsung       7733328
apple         6213902
xiaomi        4138112
huawei        1384154
lg            1024251
               ...   
kolpasan            1
emily               1
zazu                1
powerplant          1
invotone            1
Name: count, Length: 1987, dtype: int64



In [15]:
visitor_id_counts = df_cleaned['Visitor_ID'].value_counts()
visitor_id_counts

Visitor_ID
569335945    12644
568778435     8829
512365995     5982
512475445     4799
568804062     4179
             ...  
566542118        1
517039044        1
573045499        1
573047848        1
579969851        1
Name: count, Length: 2862935, dtype: int64

In [16]:
# Get unique user IDs with "view" events
view_users = df_cleaned[df_cleaned['Visitor_Action'] == 'view']['Visitor_ID'].unique()

# Get unique user IDs with "cart" or "purchase" events
cart_purchase_users = df_cleaned[df_cleaned['Visitor_Action'].isin(['cart', 'purchase'])]['Visitor_ID'].unique()

# Find user IDs with both "view" and "cart" or "purchase" events
view_and_cart_purchase_users = set(view_users) & set(cart_purchase_users)

# Filter the dataset to include only rows for user IDs with both "view" and "cart" or "purchase" events
filtered_df = df_cleaned[df_cleaned['Visitor_ID'].isin(view_and_cart_purchase_users)]

filtered_df = filtered_df.reset_index(drop=True)

# Print the filtered DataFrame
filtered_df

,Timestamp,Day_of_Week,Day_of_Month,Visitor_ID,Visitor_Action,Product,Product_Category,Category_Name,Brand,Price
0,2019-11-01 00:00:01+00:00,4,1,518085591,view,3601530,2053013563810775923,appliances.kitchen.washer,lg,713.0
1,2019-11-01 00:00:02+00:00,4,1,532647354,view,1004258,2053013555631882655,electronics.smartphone,apple,732.0
2,2019-11-01 00:00:05+00:00,4,1,526595547,view,4600658,2053013563944993659,appliances.kitchen.dishwasher,samsung,412.0
3,2019-11-01 00:00:10+00:00,4,1,513741043,view,1307350,2053013558920217191,computers.notebook,acer,515.0
4,2019-11-01 00:00:14+00:00,4,1,549256216,view,1002544,2053013555631882655,electronics.smartphone,apple,458.0
...,...,...,...,...,...,...,...,...,...,...
23772862,2019-11-30 23:59:49+00:00,5,30,514720898,view,1801141,2053013554415534427,electronics.video.tv,yasin,252.0
23772863,2019-11-30 23:59:53+00:00,5,30,513558661,view,4804294,2053013554658804075,electronics.audio.headphone,meizu,64.0
23772864,2019-11-30 23:59:58+00:00,5,30,545223467,view,28719425,2053013565639492569,apparel.shoes,baden,63.0
23772865,2019-11-30 23:59:59+00:00,5,30,557794415,view,1004833,2053013555631882655,electronics.smartphone,samsung,167.0


In [17]:
visitor_id_counts = filtered_df['Visitor_ID'].value_counts()
visitor_id_counts

Visitor_ID
568778435    8829
512365995    5982
512475445    4799
568804062    4179
512388419    3646
             ... 
541717224       2
567130741       2
571499809       2
568961363       2
579967455       2
Name: count, Length: 648550, dtype: int64

In [18]:
filtered_df['Visitor_Action'].value_counts()

Visitor_Action
view        20999670
cart         2114281
purchase      658916
Name: count, dtype: int64

In [19]:
# Check the memory usage of the filtered DataFrame in MB
memory_usage_mb = filtered_df.memory_usage(deep=True).sum() / (1024 * 1024)
print("Memory usage of filtered_df:", memory_usage_mb, "MB")


Memory usage of filtered_df: 5699.100270271301 MB


In [20]:
# Create a dictionary to map original category codes to their respective numeric labels
category_mapping = {category: label + 1 for label, category in enumerate(filtered_df['Product_Category'].unique())}

# Map the original category codes to their respective numeric labels and replace the values in the 'Product_Category' column
filtered_df['Product_Category'] = filtered_df['Product_Category'].map(category_mapping)

# Create a dictionary to map original Visitor IDs to their respective numeric labels
visitor_mapping = {visitor_id: label + 1 for label, visitor_id in enumerate(filtered_df['Visitor_ID'].unique())}

# Replace the original Visitor IDs with their respective numeric labels in the 'Visitor_ID' column
filtered_df['Visitor_ID'] = filtered_df['Visitor_ID'].map(visitor_mapping)

# Create a dictionary to map original products to their respective numeric labels
product_mapping = {product: label + 1 for label, product in enumerate(filtered_df['Product'].unique())}

# Replace the original products with their respective numeric labels in the 'Product' column
filtered_df['Product'] = filtered_df['Product'].map(product_mapping)

# Print the DataFrame with the replaced values
filtered_df


,Timestamp,Day_of_Week,Day_of_Month,Visitor_ID,Visitor_Action,Product,Product_Category,Category_Name,Brand,Price
0,2019-11-01 00:00:01+00:00,4,1,1,view,1,1,appliances.kitchen.washer,lg,713.0
1,2019-11-01 00:00:02+00:00,4,1,2,view,2,2,electronics.smartphone,apple,732.0
2,2019-11-01 00:00:05+00:00,4,1,3,view,3,3,appliances.kitchen.dishwasher,samsung,412.0
3,2019-11-01 00:00:10+00:00,4,1,4,view,4,4,computers.notebook,acer,515.0
4,2019-11-01 00:00:14+00:00,4,1,5,view,5,2,electronics.smartphone,apple,458.0
...,...,...,...,...,...,...,...,...,...,...
23772862,2019-11-30 23:59:49+00:00,5,30,359900,view,11636,18,electronics.video.tv,yasin,252.0
23772863,2019-11-30 23:59:53+00:00,5,30,126280,view,148,8,electronics.audio.headphone,meizu,64.0
23772864,2019-11-30 23:59:58+00:00,5,30,53293,view,20645,91,apparel.shoes,baden,63.0
23772865,2019-11-30 23:59:59+00:00,5,30,640953,view,112,2,electronics.smartphone,samsung,167.0


In [21]:
# Group the DataFrame by 'Category_Code' and aggregate 'Category_Name' as a set
category_groups = filtered_df.groupby('Product_Category')['Category_Name'].unique()

# Print the category names with the same category code
for Product_Category, Category_Name in category_groups.items():
    print(f"Category Code: {Product_Category}, Category Names: {Category_Name}")


Category Code: 1, Category Names: ['appliances.kitchen.washer']
Category Code: 2, Category Names: ['electronics.smartphone']
Category Code: 3, Category Names: ['appliances.kitchen.dishwasher']
Category Code: 4, Category Names: ['computers.notebook']
Category Code: 5, Category Names: ['appliances.environment.water_heater']
Category Code: 6, Category Names: ['apparel.tshirt']
Category Code: 7, Category Names: ['auto.accessories.compressor']
Category Code: 8, Category Names: ['electronics.audio.headphone']
Category Code: 9, Category Names: ['appliances.kitchen.refrigerators']
Category Code: 10, Category Names: ['electronics.audio.subwoofer']
Category Code: 11, Category Names: ['construction.tools.welding']
Category Code: 12, Category Names: ['furniture.kitchen.table']
Category Code: 13, Category Names: ['construction.tools.painting']
Category Code: 14, Category Names: ['appliances.personal.massager']
Category Code: 15, Category Names: ['furniture.kitchen.chair']
Category Code: 16, Categor

In [22]:
# Step 1: Split the strings under the 'Category_Name' column by each '.'
split_columns = filtered_df['Category_Name'].str.split('.')

# Step 2: Assign a unique numerical value to each unique word
unique_words = set(word for row in split_columns for word in row)
word_to_num = {word: i+1 for i, word in enumerate(unique_words)}

# Step 3: Combine unique numerical values for each word to form a new numerical code for each initial unique full string
numerical_codes = split_columns.apply(lambda row: '.'.join(str(word_to_num[word]) for word in row))

# Add the new numerical codes as a new column in the DataFrame
filtered_df['Category_Name_Numeric'] = numerical_codes

# Print the DataFrame with the new numerical codes
filtered_df

,Timestamp,Day_of_Week,Day_of_Month,Visitor_ID,Visitor_Action,Product,Product_Category,Category_Name,Brand,Price,Category_Name_Numeric
0,2019-11-01 00:00:01+00:00,4,1,1,view,1,1,appliances.kitchen.washer,lg,713.0,43.60.104
1,2019-11-01 00:00:02+00:00,4,1,2,view,2,2,electronics.smartphone,apple,732.0,64.80
2,2019-11-01 00:00:05+00:00,4,1,3,view,3,3,appliances.kitchen.dishwasher,samsung,412.0,43.60.58
3,2019-11-01 00:00:10+00:00,4,1,4,view,4,4,computers.notebook,acer,515.0,50.89
4,2019-11-01 00:00:14+00:00,4,1,5,view,5,2,electronics.smartphone,apple,458.0,64.80
...,...,...,...,...,...,...,...,...,...,...,...
23772862,2019-11-30 23:59:49+00:00,5,30,359900,view,11636,18,electronics.video.tv,yasin,252.0,64.71.108
23772863,2019-11-30 23:59:53+00:00,5,30,126280,view,148,8,electronics.audio.headphone,meizu,64.0,64.26.68
23772864,2019-11-30 23:59:58+00:00,5,30,53293,view,20645,91,apparel.shoes,baden,63.0,110.97
23772865,2019-11-30 23:59:59+00:00,5,30,640953,view,112,2,electronics.smartphone,samsung,167.0,64.80


In [23]:
num_to_word = {v: k for k, v in word_to_num.items()}
num_to_word

{1: 'compressor',
 2: 'air_conditioner',
 3: 'anti_freeze',
 4: 'cpu',
 5: 'juicer',
 6: 'grill',
 7: 'tonometer',
 8: 'fryer',
 9: 'toilet',
 10: 'skirt',
 11: 'faucet',
 12: 'sock',
 13: 'umbrella',
 14: 'microwave',
 15: 'oven',
 16: 'peripherals',
 17: 'ballet_shoes',
 18: 'cooler',
 19: 'blanket',
 20: 'sandals',
 21: 'personal',
 22: 'skates',
 23: 'cabinet',
 24: 'generator',
 25: 'bag',
 26: 'audio',
 27: 'step_ins',
 28: 'massager',
 29: 'bathroom',
 30: 'videocards',
 31: 'coffee_machine',
 32: 'camera',
 33: 'table',
 34: 'scarf',
 35: 'projector',
 36: 'environment',
 37: 'vacuum',
 38: 'hood',
 39: 'bath',
 40: 'photo',
 41: 'steam_cooker',
 42: 'monitor',
 43: 'appliances',
 44: 'painting',
 45: 'bicycle',
 46: 'pump',
 47: 'tablet',
 48: 'dress',
 49: 'chair',
 50: 'computers',
 51: 'trousers',
 52: 'shorts',
 53: 'screw',
 54: 'iron',
 55: 'moccasins',
 56: 'meat_grinder',
 57: 'coffee_grinder',
 58: 'dishwasher',
 59: 'living_room',
 60: 'kitchen',
 61: 'shirt',
 62: '

In [24]:
filtered_df['Category_Name_Numeric'].value_counts()

Category_Name_Numeric
64.80         9559280
64.71.108     1287215
50.89         1207901
64.26.68      1090105
64.116         995302
               ...   
110.97.144        487
110.52            214
62.72.53           61
43.60.8            48
67.100.124          1
Name: count, Length: 128, dtype: int64

In [25]:
#Dropping Category_Name 
Redefined_df= filtered_df.drop(columns='Category_Name')

In [26]:

Redefined_df = Redefined_df[['Timestamp','Day_of_Week','Day_of_Month','Visitor_ID','Visitor_Action','Product','Product_Category','Category_Name_Numeric','Brand','Price']]
Redefined_df 

,Timestamp,Day_of_Week,Day_of_Month,Visitor_ID,Visitor_Action,Product,Product_Category,Category_Name_Numeric,Brand,Price
0,2019-11-01 00:00:01+00:00,4,1,1,view,1,1,43.60.104,lg,713.0
1,2019-11-01 00:00:02+00:00,4,1,2,view,2,2,64.80,apple,732.0
2,2019-11-01 00:00:05+00:00,4,1,3,view,3,3,43.60.58,samsung,412.0
3,2019-11-01 00:00:10+00:00,4,1,4,view,4,4,50.89,acer,515.0
4,2019-11-01 00:00:14+00:00,4,1,5,view,5,2,64.80,apple,458.0
...,...,...,...,...,...,...,...,...,...,...
23772862,2019-11-30 23:59:49+00:00,5,30,359900,view,11636,18,64.71.108,yasin,252.0
23772863,2019-11-30 23:59:53+00:00,5,30,126280,view,148,8,64.26.68,meizu,64.0
23772864,2019-11-30 23:59:58+00:00,5,30,53293,view,20645,91,110.97,baden,63.0
23772865,2019-11-30 23:59:59+00:00,5,30,640953,view,112,2,64.80,samsung,167.0


In [27]:
# Define mapping of original values of Visitor_Action to unique numerical values
action_mapping = {'view': 1, 'cart': 2, 'purchase': 3}

# Replace values in the 'Visitor_Action' column with numerical values
Redefined_df['Visitor_Action'] = Redefined_df['Visitor_Action'].map(action_mapping)

# Print the DataFrame with numerical values
Redefined_df

,Timestamp,Day_of_Week,Day_of_Month,Visitor_ID,Visitor_Action,Product,Product_Category,Category_Name_Numeric,Brand,Price
0,2019-11-01 00:00:01+00:00,4,1,1,1,1,1,43.60.104,lg,713.0
1,2019-11-01 00:00:02+00:00,4,1,2,1,2,2,64.80,apple,732.0
2,2019-11-01 00:00:05+00:00,4,1,3,1,3,3,43.60.58,samsung,412.0
3,2019-11-01 00:00:10+00:00,4,1,4,1,4,4,50.89,acer,515.0
4,2019-11-01 00:00:14+00:00,4,1,5,1,5,2,64.80,apple,458.0
...,...,...,...,...,...,...,...,...,...,...
23772862,2019-11-30 23:59:49+00:00,5,30,359900,1,11636,18,64.71.108,yasin,252.0
23772863,2019-11-30 23:59:53+00:00,5,30,126280,1,148,8,64.26.68,meizu,64.0
23772864,2019-11-30 23:59:58+00:00,5,30,53293,1,20645,91,110.97,baden,63.0
23772865,2019-11-30 23:59:59+00:00,5,30,640953,1,112,2,64.80,samsung,167.0


In [28]:
# Extract hour component from 'Timestamp' column
Redefined_df['Hour_of_Day'] = Redefined_df['Timestamp'].dt.hour

Resized_df= Redefined_df.drop(columns='Timestamp')
Resized_df = Resized_df[['Hour_of_Day','Day_of_Week','Day_of_Month','Visitor_ID','Visitor_Action','Product','Product_Category','Category_Name_Numeric','Brand','Price']]
Resized_df 

,Hour_of_Day,Day_of_Week,Day_of_Month,Visitor_ID,Visitor_Action,Product,Product_Category,Category_Name_Numeric,Brand,Price
0,0,4,1,1,1,1,1,43.60.104,lg,713.0
1,0,4,1,2,1,2,2,64.80,apple,732.0
2,0,4,1,3,1,3,3,43.60.58,samsung,412.0
3,0,4,1,4,1,4,4,50.89,acer,515.0
4,0,4,1,5,1,5,2,64.80,apple,458.0
...,...,...,...,...,...,...,...,...,...,...
23772862,23,5,30,359900,1,11636,18,64.71.108,yasin,252.0
23772863,23,5,30,126280,1,148,8,64.26.68,meizu,64.0
23772864,23,5,30,53293,1,20645,91,110.97,baden,63.0
23772865,23,5,30,640953,1,112,2,64.80,samsung,167.0


In [29]:
# Check the memory usage of the filtered DataFrame in MB
memory_usage_mb = Resized_df.memory_usage(deep=True).sum() / (1024 * 1024)
print("Memory usage of Resized_df:", memory_usage_mb, "MB")


Memory usage of Resized_df: 4043.1280250549316 MB


In [30]:
Resized_df['Product'].value_counts()

Product
59       424462
84       406393
122      351898
172      281333
202      199696
          ...  
58197         1
58194         1
45856         1
58191         1
64095         1
Name: count, Length: 64095, dtype: int64

In [31]:
#Filter the DataFrame to include only "view" events
view_df = Resized_df[Resized_df['Visitor_Action'] == 1]

# Group by 'Visitor_ID' and count the number of products viewed by each visitor
products_viewed_by_visitor = view_df.groupby('Visitor_ID')['Product'].count()

# Sum the counts of products viewed by all visitors
total_products_viewed = products_viewed_by_visitor.sum()

# Get the total number of unique visitors
total_visitors = Resized_df['Visitor_ID'].nunique()

# Calculate the formula
Avg_P_W_V = total_products_viewed / total_visitors

print("Total products viewed:", total_products_viewed)
print("Total number of visitors:", total_visitors)
print("Average products viewed per visitor:", Avg_P_W_V)

Total products viewed: 20999670
Total number of visitors: 648550
Average products viewed per visitor: 32.37941561945879


In [32]:
# Count the number of apprearance of each product
product_counts = Resized_df['Product'].value_counts()

# Filter out products that have been appeared an average of 35 times throughout the month
products_to_keep = product_counts[product_counts > 34].index

#Filter the original DataFrame
Resized_filt_df = Resized_df[Resized_df['Product'].isin(products_to_keep)]

Resized_filt_df['Product'].value_counts()

Product
59       424462
84       406393
122      351898
172      281333
202      199696
          ...  
52806        35
23025        35
23032        35
3495         35
14816        35
Name: count, Length: 30191, dtype: int64

In [33]:
Resized_filt_df = Resized_filt_df.reset_index(drop=True)
Resized_filt_df

,Hour_of_Day,Day_of_Week,Day_of_Month,Visitor_ID,Visitor_Action,Product,Product_Category,Category_Name_Numeric,Brand,Price
0,0,4,1,1,1,1,1,43.60.104,lg,713.0
1,0,4,1,2,1,2,2,64.80,apple,732.0
2,0,4,1,3,1,3,3,43.60.58,samsung,412.0
3,0,4,1,4,1,4,4,50.89,acer,515.0
4,0,4,1,5,1,5,2,64.80,apple,458.0
...,...,...,...,...,...,...,...,...,...,...
23394187,23,5,30,359900,1,11636,18,64.71.108,yasin,252.0
23394188,23,5,30,126280,1,148,8,64.26.68,meizu,64.0
23394189,23,5,30,53293,1,20645,91,110.97,baden,63.0
23394190,23,5,30,640953,1,112,2,64.80,samsung,167.0


In [44]:
# Filter the DataFrame to include only "purchase" events
purchase_df = Resized_filt_df[Resized_filt_df['Visitor_Action'] == 3]

# Calculate total revenue by summing up the 'Price' values for purchase events
total_revenue = purchase_df['Price'].sum()

print("Total revenue:", total_revenue)

# Calculate the average product price across all products
average_product_price = Resized_filt_df['Price'].mean()

print("Average Product Price Across All Products:", average_product_price)


Total revenue: 241105765.0
Average Product Price Across All Products: 357.9878592943069


In [35]:
# Filter the DataFrame for rows where Price is between 1 and 9 and Visitor_Action is 'purchase'
P_1_9 = Resized_filt_df[(Resized_filt_df['Price'] >= 1) & (Resized_filt_df['Price'] <= 9) & (Resized_filt_df['Visitor_Action'] == 3)]
# Calculate the total revenue from the filtered DataFrame
P_1_9_tot = P_1_9['Price'].sum()
print("Total contribution of the purchased product where Price is between 1 and 9 to total revenue is:", P_1_9_tot,"at" ,round(P_1_9_tot/total_revenue*100, 3), "%")

Total contribution of the purchased product where Price is between 1 and 9 to total revenue is: 27979.0 at 0.012 %


In [36]:
# Filter the original DataFrame to exclude rows in P_1_9
Resized_filt2_df = Resized_filt_df[Resized_filt_df['Price'] > 9]
Resized_filt2_df = Resized_filt2_df.reset_index(drop=True)
Resized_filt2_df

,Hour_of_Day,Day_of_Week,Day_of_Month,Visitor_ID,Visitor_Action,Product,Product_Category,Category_Name_Numeric,Brand,Price
0,0,4,1,1,1,1,1,43.60.104,lg,713.0
1,0,4,1,2,1,2,2,64.80,apple,732.0
2,0,4,1,3,1,3,3,43.60.58,samsung,412.0
3,0,4,1,4,1,4,4,50.89,acer,515.0
4,0,4,1,5,1,5,2,64.80,apple,458.0
...,...,...,...,...,...,...,...,...,...,...
23237680,23,5,30,359900,1,11636,18,64.71.108,yasin,252.0
23237681,23,5,30,126280,1,148,8,64.26.68,meizu,64.0
23237682,23,5,30,53293,1,20645,91,110.97,baden,63.0
23237683,23,5,30,640953,1,112,2,64.80,samsung,167.0


In [37]:

Resized_filt2_df['Price'].max()

2574.0

In [38]:
Resized_filt2_df.dtypes

Hour_of_Day                int32
Day_of_Week                int32
Day_of_Month               int32
Visitor_ID                 int64
Visitor_Action             int64
Product                    int64
Product_Category           int64
Category_Name_Numeric     object
Brand                     object
Price                    float64
dtype: object

In [39]:
#Making datatype more efficient in terms of storage space 
# Convert Hour_of_Day, Day_of_Week, Day_of_Month to uint8
Resized_filt2_df['Hour_of_Day'] = Resized_filt2_df['Hour_of_Day'].astype('uint8')
Resized_filt2_df['Day_of_Week'] = Resized_filt2_df['Day_of_Week'].astype('uint8')
Resized_filt2_df['Day_of_Month'] = Resized_filt2_df['Day_of_Month'].astype('uint8')

# Convert Visitor_Action to uint8 since it has only 3 unique values
Resized_filt2_df['Visitor_Action'] = Resized_filt2_df['Visitor_Action'].astype('uint8')

# Convert Visitor_ID, Product, Product_Category to uint32
Resized_filt2_df['Visitor_ID'] = Resized_filt2_df['Visitor_ID'].astype('uint32')
Resized_filt2_df['Product'] = Resized_filt2_df['Product'].astype('uint32')
Resized_filt2_df['Product_Category'] = Resized_filt2_df['Product_Category'].astype('uint32')

# Convert Price to uint16 since it has values up to 2574
Resized_filt2_df['Price'] = Resized_filt2_df['Price'].astype('uint16')

# Convert Category_Name_Numeric and Brand to category type
Resized_filt2_df['Category_Name_Numeric'] = Resized_filt2_df['Category_Name_Numeric'].astype('category')
Resized_filt2_df['Brand'] = Resized_filt2_df['Brand'].astype('category')

# Print the updated data types
Resized_filt2_df.dtypes

Hour_of_Day                 uint8
Day_of_Week                 uint8
Day_of_Month                uint8
Visitor_ID                 uint32
Visitor_Action              uint8
Product                    uint32
Product_Category           uint32
Category_Name_Numeric    category
Brand                    category
Price                      uint16
dtype: object

In [40]:
# Check the memory usage of the filtered DataFrame in MB
memory_usage_mb = Resized_filt2_df.memory_usage(deep=True).sum() / (1024 * 1024)
print("Memory usage of Resized_filt2_df:", memory_usage_mb, "MB")

Memory usage of Resized_filt2_df: 465.5119981765747 MB


In [41]:
# Save the modified DataFrame to a new CSV file
Resized_filt2_df.to_csv('Resources/DF_Resized.csv', index=False, encoding='utf-8')


In [42]:
#Calculating KPIs and analysis based on final cleaned DF

#Filter the DataFrame to include only "view" events
view_df2 = Resized_filt2_df[Resized_filt2_df['Visitor_Action'] == 1]

# Group by 'Visitor_ID' and count the number of products viewed by each visitor
products_viewed_by_visitor2 = view_df2.groupby('Visitor_ID')['Product'].count()

# Sum the counts of products viewed by all visitors
total_products_viewed2 = products_viewed_by_visitor2.sum()

# Get the total number of unique visitors
total_visitors2 = Resized_filt2_df['Visitor_ID'].nunique()

# Calculate the formula
Avg_P_W_V2 = np.ceil(total_products_viewed2 / total_visitors2)

# Filter the DataFrame to include only "purchase" events
purchase_df2 = Resized_filt2_df[Resized_filt2_df['Visitor_Action'] == 3]
total_orders = len(purchase_df2)

# Calculate total revenue by summing up the 'Price' values for purchase events
total_revenue2 = purchase_df2['Price'].sum()

# Calculate the average product price across all products
aop = np.ceil(Resized_filt2_df['Price'].mean())

# Calculate the Average Order Value (AOV)
aov = np.ceil((total_revenue2 / total_orders) * 100)

# Calculate the number of unique users who made a purchase
num_purchases = purchase_df2['Visitor_ID'].nunique()

# Calculate the Conversion Rate (CVR %) 
cvr = np.ceil((num_purchases / total_visitors2) * 100)

# Group the DataFrame by visitor_id and count the number of purchases
purchase_counts = purchase_df2.groupby('Visitor_ID').size()

# Filter the DataFrame to include only visitor_id with two or more purchases
return_clients = purchase_counts[purchase_counts >= 2]

# Calculate the percentage of return clients
percentage_return_clients = np.ceil((len(return_clients) / Resized_filt2_df['Visitor_ID'].nunique()) * 100)

# Filter the DataFrame to include only events related to shopping carts created
cart_created_df = Resized_filt2_df[Resized_filt2_df['Visitor_Action'] == 2]

# Count the total number of shopping carts created
num_shopping_carts_created = len(cart_created_df)

# Calculate the Shopping Cart Abandonment Rate
shopping_cart_abandonment_rate = np.ceil((total_orders / num_shopping_carts_created) * 100)

print("Total products viewed:", total_products_viewed2)
print("Total number of visitors:", total_visitors2)
print("Total number of orders:", total_orders)
print("Average products viewed per visitor:", Avg_P_W_V2)
print("Total revenue:", total_revenue2)
print("Average Product Price Across All Products:", aop)
print("Average Value of Purchase Transaction:", aov)
print("Conversion Rate (CVR):", cvr)
print("Shopping Cart Abandonment Rate:", shopping_cart_abandonment_rate)
print("Percentage of return clients with two or more purchases:", percentage_return_clients)

Total products viewed: 20493555
Total number of visitors: 647455
Total number of orders: 652966
Average products viewed per visitor: 32.0
Total revenue: 241077786
Average Product Price Across All Products: 361.0
Average Value of Purchase Transaction: 36921.0
Conversion Rate (CVR): 51.0
Shopping Cart Abandonment Rate: 32.0
Percentage of return clients with two or more purchases: 19.0


In [43]:

# Top 5 Highest Grossing Products

grossing_products = purchase_df2.groupby('Product')['Price'].sum().sort_values(ascending=False).head(5)

# Top 5 Products by Conversion Rates
purchase_counts = purchase_df2.groupby('Product').size()
view_counts = Resized_filt2_df[Resized_filt2_df['Visitor_Action'] == 1].groupby('Product').size()
Top10_CVR = (purchase_counts / view_counts).sort_values(ascending=False).head(5)

# Group the DataFrame by product category and sum the prices to calculate total revenue for each category
category_revenue = Resized_filt2_df.groupby('Product_Category')['Price'].sum().reset_index()

# Sort the categories by total revenue in descending order
top_5_categories = category_revenue.sort_values(by='Price', ascending=False).head(5)
print("\nTop 5 Highest Grossing Products:")
print(grossing_products)
print("\nTop 5 Products by Conversion Rates:")
print(Top10_CVR)
print("\nTop 5 Product Categories by Total Revenue:")
print(top_5_categories)



Top 5 Highest Grossing Products:
Product
59     20606392
124    11437701
53      7079799
190     6813467
5       5597573
Name: Price, dtype: uint64

Top 5 Products by Conversion Rates:
Product
40688    0.276923
5272     0.250000
4816     0.238095
53028    0.217391
57671    0.212121
dtype: float64

Top 5 Product Categories by Total Revenue:
    Product_Category       Price
1                  2  4574324905
3                  4   850192043
17                18   586426534
0                  1   274956903
8                  9   231766668


In [47]:

# Filter the main DataFrame to include only products belonging to the top 5 categories
top_products = Resized_filt2_df[Resized_filt2_df['Product_Category'].isin(top_5_categories['Product_Category'])]

# Group the top products DataFrame by product category and sort each group by price to get top products within each category
top_products = top_products.groupby('Product_Category').apply(lambda x: x.nlargest(1, 'Price')).reset_index(drop=True)

# Reverse the mapping dictionary to map numerical codes back to original Category_Name strings
num_to_category_name = {v: k for k, v in word_to_num.items()}

# Retrieve the original Category_Name associated with the top products
top_products['Category_Name'] = top_products['Category_Name_Numeric'].apply(lambda x: '.'.join(num_to_category_name[int(num)] for num in x.split('.')))

# Print the top products with their associated original Category_Name
print(top_products[['Product', 'Category_Name']])

   Product                     Category_Name
0    34038         appliances.kitchen.washer
1    42732            electronics.smartphone
2    11588                computers.notebook
3    17275  appliances.kitchen.refrigerators
4     1580              electronics.video.tv


/var/folders/bf/0vmhxl6x4xn54k_fw54k_rkr0000gn/T/ipykernel_50134/3891696463.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  top_products = top_products.groupby('Product_Category').apply(lambda x: x.nlargest(1, 'Price')).reset_index(drop=True)


In [55]:
# Group by Hour_of_Day, Day_of_Week, and Day_of_Month for views
hourly_views = view_df2.groupby('Hour_of_Day').size()
daily_views = view_df2.groupby('Day_of_Week').size()
monthly_views = view_df2.groupby('Day_of_Month').size()

# Group by Hour_of_Day, Day_of_Week, and Day_of_Month for purchases
hourly_purchases = purchase_df2.groupby('Hour_of_Day').size()
daily_purchases = purchase_df2.groupby('Day_of_Week').size()
monthly_purchases = purchase_df2.groupby('Day_of_Month').size()

# Find the top 3 hour, day of the week, and day of the month with the highest number of views and purchases
max_hourly_views = hourly_views.nlargest(3)
max_daily_views = daily_views.nlargest(3)
max_monthly_views = monthly_views.nlargest(3)

max_hourly_purchases = hourly_purchases.nlargest(3)
max_daily_purchases = daily_purchases.nlargest(3)
max_monthly_purchases = monthly_purchases.nlargest(3)

# Print results
print(f"Hour of the day with the highest number of views: {max_hourly_views}")
print(f"Day of the week with the highest number of views: {max_daily_views}")
print(f"Day of the month with the highest number of views: {max_monthly_views}")
print(f"Hour of the day with the highest number of purchases: {max_hourly_purchases}")
print(f"Day of the week with the highest number of purchases: {max_daily_purchases}")
print(f"Day of the month with the highest number of purchases: {max_monthly_purchases}")

Hour of the day with the highest number of views: Hour_of_Day
16    1314042
17    1307106
15    1297075
dtype: int64
Day of the week with the highest number of views: Day_of_Week
4    4387365
5    4212072
6    3705646
dtype: int64
Day of the month with the highest number of views: Day_of_Month
15    2616329
16    2473428
17    2362833
dtype: int64
Hour of the day with the highest number of purchases: Hour_of_Day
9     51634
10    50021
8     46856
dtype: int64
Day of the week with the highest number of purchases: Day_of_Week
6    181946
5    119353
4     75510
dtype: int64
Day of the month with the highest number of purchases: Day_of_Month
17    133602
16     50865
29     24049
dtype: int64


In [58]:
import sqlalchemy
from sqlalchemy import create_engine, text, func
from sqlalchemy import Column, Integer, String, Float
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import Session
from sqlalchemy.ext.automap import automap_base
from sqlalchemy import inspect
import altair as alt
import json
import os
